In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('./data/statlog_heart/heart_data.csv')
df = df.iloc[:100]


In [2]:
df = df.rename(columns={'heart_disease': 'target'})
X = df.drop('target', axis=1)
y = df['target']
print(X.shape, y.shape)

(100, 13) (100,)


In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-np.pi, np.pi))
X = scaler.fit_transform(X)
y = np.array(y).reshape(-1, 1)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.33, random_state=42)

print("Data: ", X.shape, y.shape)
print("Train set:", X_train.shape, y_train.shape)
print("Validation set:", X_valid.shape, y_valid.shape)
print("Test set:", X_test.shape, y_test.shape)

Data:  (100, 13) (100, 1)
Train set: (70, 13) (70, 1)
Validation set: (20, 13) (20, 1)
Test set: (10, 13) (10, 1)


In [5]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.algorithms import VQC
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit_algorithms.optimizers import COBYLA

In [6]:
zz_feature_map = ZZFeatureMap(
    feature_dimension=X_train.shape[1], 
    reps=2)
pauli_feature_map = PauliFeatureMap(
    feature_dimension=X_train.shape[1], 
    reps=2, paulis=["Z", "YY"], 
    entanglement="full"
)


In [7]:
from src.experiment import Experiment

experiment1 = Experiment(
    X_train, y_train, 
    X_valid, y_valid, 
    X_test, y_test, 
    num_qubits=X_train.shape[1]
)

experiment2 = Experiment(
    X_train, y_train, 
    X_valid, y_valid, 
    X_test, y_test, 
    num_qubits=X_train.shape[1]
)

In [8]:
from src.ansatz import *

In [28]:
# print(zz_feature_map._base_name)
# print(pauli_feature_map._base_name)

ZZFeatureMap
PauliFeatureMap


In [9]:
print("[+] Running experiment with ZZ Feature Map...")
experiment1.run_experiment(zz_feature_map, [ansatz_1, ansatz_2, ansatz_3, ansatz_4], max_reps=3)
experiment2.plot_results()

test_results = experiment1.evaluate_best_on_test()
print("Test Results for the Best Model on ZZ Feature Set:", test_results)



[+] Running experiment with ZZ Feature Map...


AttributeError: 'ZZFeatureMap' object has no attribute '__name__'

In [ ]:
print("[+] Running experiment with Pauli Feature Map...")
experiment2.run_experiment(pauli_feature_map, [ansatz_1, ansatz_2, ansatz_3, ansatz_4], max_reps=4)
experiment2.plot_results()

test_results = experiment2.evaluate_best_on_test()
print("Test Results for the Best Model on Pauli Feature Set:", test_results)